# Julia for Economists

## Code for the Go-Fast

### Cameron Pfiffer (cpfiffer@stanford.edu)

## Introduction

- I'm Cameron Pfiffer, a finance PhD candidate from the University of Oregon.
- Visiting at Stanford
- This is the fourth in a series of workshops on Julia for economists
- Past lectures/recordings/notes are in the GitHub repo: https://github.com/cpfiffer/julia-bootcamp-2022

## Format

- The sessions are two hours now! 
- Today is all lecture! There's going to be a ton of examples for us to walk through together.
- Ask questions when needed -- please interrupt me!
- I assume you have Julia and some text editor evailable to you. If you don't have it, please download a Julia binary here: https://julialang.org/downloads/

## Topic map

- What the heck is a compiler?
- Performant Julia! How do you write Julia code that (reliably) goes fast?
- Tracking memory allocation
- Tackling type instability
- Closures
- Profiling & code analysis tools (Traceur, Profile, ProfileViews, BenchmarkTools, etc.)

## Citations

Many of these tips come from the excellent [Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/) guide on the Julia docs.

## The Julia compiler

A _compiler_ is a thing that takes human-readable code and turns it into machine-readable code. Let's take a look at one way that the compiler does this by making a really simple function to increment a value by 1.

In [5]:
# Write test_function
test_function(x) = x + 1

test_function (generic function with 1 method)

## The `code_typed` macro

`code_typed` takes a function call and returns the Julia-typed function. This is a good way to analyze what Julia thinks about your function! More on its use later.

In [6]:
@code_typed test_function(1.0)

CodeInfo(
1 ─ %1 = Base.add_float(x, 1.0)::Float64
└──      return %1
) => Float64

Julia will take this and give it to [LLVM](https://en.wikipedia.org/wiki/LLVM), which is the next level down in the compiler chain. LLVM is a language-independent compiler architecture tool that is used by many languages, not just Julia.

In [7]:
@code_llvm test_function(1.0)

;  @ In[5]:2 within `test_function`
define double @julia_test_function_1911(double %0) #0 {
top:
; ┌ @ promotion.jl:379 within `+` @ float.jl:399
   %1 = fadd double %0, 1.000000e+00
; └
  ret double %1
}


Lastly, the LLVM code goes to native code, which is machine code specialized to your system architecture:

In [8]:
@code_native test_function(1.0)

	.text
; ┌ @ In[5]:2 within `test_function`
	movabsq	$.rodata.cst8, %rax
; │┌ @ promotion.jl:379 within `+` @ float.jl:399
	vaddsd	(%rax), %xmm0, %xmm0
; │└
	retq
	nop
; └


## Specialization -- different inputs, different code!

In [10]:
@code_native debuginfo=:none test_function(1.0)

	.text
	movabsq	$.rodata.cst8, %rax
	vaddsd	(%rax), %xmm0, %xmm0
	retq
	nop


In [14]:
@code_native debuginfo=:none test_function(1)

	.text
	leaq	1(%rdi), %rax
	retq
	nopw	%cs:(%rax,%rax)


This gets ridiculous if you get outside numeric types:

In [17]:
@code_typed debuginfo=:none test_function('a')

CodeInfo(
1 ──        goto #25 if not true
2 ── %2   = Base.bitcast(Base.UInt32, x)::UInt32
│    %3   = Base.lshr_int(%2, 0x0000000000000018)::UInt32
│    %4   = Base.shl_int(%2, 0xffffffffffffffe8)::UInt32
│    %5   = Base.ifelse(true, %3, %4)::UInt32
│    %6   = Base.trunc_int(Int8, %5)::Int8
│    %7   = Core.sext_int(Core.Int32, %6)::Int32
│    %8   = Core.sext_int(Core.Int64, %7)::Int64
│    %9   = Base.sle_int(0, %8)::Bool
└───        goto #25 if not %9
3 ── %11  = Base.sext_int(Int64, %7)::Int64
│    %12  = Base.add_int(%11, 1)::Int64
│    %13  = Base.sle_int(0, %12)::Bool
└───        goto #5 if not %13
4 ── %15  = Base.slt_int(%12, 0)::Bool
│    %16  = Base.bitcast(UInt64, %12)::UInt64
│    %17  = Base.ult_int(%16, 0x0000000000000080)::Bool
│    %18  = Base.or_int(%15, %17)::Bool
└───        goto #6
5 ──        nothing::Nothing
6 ┄─ %21  = φ (#4 => %18, #5 => false)::Bool
└───        goto #8 if not %21
7 ── %23  = Base.trunc_int(UInt32, %12)::UInt32
│    %24  = Base.shl_int(%23,

## Why am I telling you this?

Every time you give a function a new type of input, Julia will compile a new specialization! 

If the new type is concrete and precise, Julia can make a precise, fast, specialized function.

If the new type is abstract and general, Julia can't do that specialization. It can't make ultra-fast native code for you!

## Type stability

This problem is known as __type stability__. When you call the same function a lot of times with different inputs, Julia has to compile a bunch of specialized code to handle it!

This is slow, not only because compiling can be slow, but also because type-unstable functions do not benefit from the amazing compiler optimizations that you can do when you know the type of everything.

## Type stability: an example


In [9]:
bad_function(x) = x >= 5 ? 'a' : 5*2
@code_warntype bad_function(3)

MethodInstance for bad_function(::Int64)
  from bad_function(x) in Main at In[9]:1
Arguments
  #self#::Core.Const(bad_function)
  x::Int64
Body::Union{Char, Int64}
1 ─ %1 = (x >= 5)::Bool
└──      goto #3 if not %1
2 ─      return 'a'
3 ─ %4 = (5 * 2)::Core.Const(10)
└──      return %4



Type instability can end up being a huge pain if it appears __anywhere__ in another function and affects the return type of an outer function:

In [10]:
function nested_function(y)
    x = bad_function(y)
    return x + 1
end

@code_warntype nested_function(1.0)

MethodInstance for nested_function(::Float64)
  from nested_function(y) in Main at In[10]:1
Arguments
  #self#::Core.Const(nested_function)
  y::Float64
Locals
  x::Union{Char, Int64}
Body::Union{Char, Int64}
1 ─      (x = Main.bad_function(y))
│   %2 = (x + 1)::Union{Char, Int64}
└──      return %2



## Type instability

Type stability is loosely defined as when the type of a function's output is only defined by the __type__ of the input, and not the __value__ of the input.

When Julia doesn't know the concrete type of a thing, the code touching that thing gets worse.

In [13]:
maybe_zero(x) = x >= 0 ? 0 : x
@code_warntype maybe_zero(1.0)

MethodInstance for maybe_zero(::Float64)
  from maybe_zero(x) in Main at In[13]:1
Arguments
  #self#::Core.Const(maybe_zero)
  x::Float64
Body::Union{Float64, Int64}
1 ─ %1 = (x >= 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x



When we call this a bunch of times, the type instability _really_ starts to kick in:

In [14]:
using BenchmarkTools
xs = randn(1_000)
@benchmark map(maybe_zero, $xs)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  11.419 μs …  1.110 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     24.740 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   26.257 μs ± 16.581 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

             ▃█▁                                               
  ▂▂▂▁▁▁▁▁▁▂▄███▆▅▄▃▄▄▄▄▃▃▃▃▃▃▄▅▆▆▇▇▆▅▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  11.4 μs         Histogram: frequency by time        46.4 μs <

 Memory estimate: 24.03 KiB, allocs estimate: 524.

Let's fix it by making sure that the type of the output is the same regardless of whether $x \ge 0$. A handy function is `zero(thing)`, which will attempt to return a value of zero in the same type as `thing`.

In [17]:
better_zero(x) = x >= 0 ? zero(x) : x
@code_warntype better_zero(1.0)

MethodInstance for better_zero(::Float64)
  from better_zero(x) in Main at In[17]:1
Arguments
  #self#::Core.Const(better_zero)
  x::Float64
Body::Float64
1 ─ %1 = (x >= 0)::Bool
└──      goto #3 if not %1
2 ─ %3 = Main.zero(x)::Core.Const(0.0)
└──      return %3
3 ─      return x



Now, see how much faster this is! Fewer allocations, more consistent timing, and lower median/mean times.

In [18]:
@benchmark map(better_zero, $xs)

BenchmarkTools.Trial: 10000 samples with 26 evaluations.
 Range (min … max):  865.577 ns … 435.081 μs  ┊ GC (min … max): 0.00% … 98.51%
 Time  (median):       1.389 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.975 μs ±   8.196 μs  ┊ GC (mean ± σ):  8.97% ±  2.19%

  ▆▇██▇▇▆▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁                     ▁▁▁▁               ▃
  ██████████████████████████▇▇▆▇▇█▇█▇▇▇▆▇▆██▇███████▇█▇▇▆▆▅▇▅▆▆ █
  866 ns        Histogram: log(frequency) by time       6.84 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

## How to handle type instability

- Use `@code_warntype` to chase down type instabilities.
- Use `zero(returntype)` to get correct zero for your type
- With arrays, use `zeros(typeof(thing), n)`


## Traceur.jl

A _really_ good tool to use is `Traceur`, a package which flags a bunch of common performance issues in Julia on your behalf. 

To use it, just call `@trace` right before your function call and it'll give you a detailed report.

Install it with `] add Traceur` or `import Pkg; Pkg.add("Traceur")`.

In [26]:
using Traceur

# Define a type unstable function
me_badman(x) = x <= 5 ? convert(typeof(x), 5) : x
# me_badman(x::T) where T = x <= 5 ? convert(T, 5) : x # Type stable
@trace me_badman(5.0)

5.0

## Concrete typing of structs and arrays

There are a bunch of super types that are intended to describe lots of types of variables. For example, the `String` `"howdy partner"` and the `Int` `12` are both of subtype `Any`.

When we put them together in an array, the array has to have an element type that all elements are subtypes of.

In [28]:
# Create an array with lots of different types
# Note that the element type here is Any!
arr = [12, "howdy partner", 3.9]
eltype(arr)
arr

3-element Vector{Any}:
 12
   "howdy partner"
  3.9

## Abstract typed arrays are bad

Arrays that are not specifically typed can be _really inefficient_, because Julia has to alocate memory assuming you can put literally anything in them. 

If you tell it an array will _only_ hold `Float64`, it can assign 64-bit storage for each element inside it, and you benefit from all kinds of fancy math specializations.

In [30]:
# Bad and slow array of `Real`s
xs = randn(300, 300)
matrix = Matrix{Real}(xs)
@time matrix * matrix;

# Good and fast array of Float64
matrix = Matrix{Float64}(xs)
@time matrix * matrix;

  2.840901 seconds (54.27 M allocations: 828.781 MiB, 9.29% gc time)
  0.002297 seconds (2 allocations: 703.172 KiB)


You might be thinking 

> Gosh Cameron, I'd never do this! I'm a top-notch code cowboy.

and you _might_ be right. 

However! I see this in others, and I often screw up and do this myself:

```julia
array = []

for something in an_iterator
    . . .
    push!(array, nifty_function(something))
end
```

What's wrong here?

![](https://c.tenor.com/m6GfTVTJqN8AAAAd/bad-bad-boy.gif)

The type of the array defaults to `Any`, because I haven't provided any other information! 

If you know for a fact it's going to be of a certain type, type it ahead of time:

```julia
array = Float64[]
```

or if you don't know it's exactly `Float64` but you have something available you know is of the type you will be putting in the array, you can do

```julia
array = typeof(a_thing)[]
```

or, if you know the size, just preallocate the whole array with

```julia
array = Array{typeof(a_thing)}(undef, size_of_array)
```

In [36]:
# Let's see an example of how to operationalize this:
do_stuff(x) = (x=x, sin_x=sin(x), str_x=string(x))
@code_warntype do_stuff(1.0)
typeof(do_stuff(1))


MethodInstance for do_stuff(::Float64)
  from do_stuff(x) in Main at In[36]:2
Arguments
  #self#::Core.Const(do_stuff)
  x::Float64
Body::NamedTuple{(:x, :sin_x, :str_x), Tuple{Float64, Float64, String}}
1 ─ %1 = (:x, :sin_x, :str_x)::Core.Const((:x, :sin_x, :str_x))
│   %2 = Core.apply_type(Core.NamedTuple, %1)::Core.Const(NamedTuple{(:x, :sin_x, :str_x)})
│   %3 = Main.sin(x)::Float64
│   %4 = Main.string(x)::String
│   %5 = Core.tuple(x, %3, %4)::Tuple{Float64, Float64, String}
│   %6 = (%2)(%5)::NamedTuple{(:x, :sin_x, :str_x), Tuple{Float64, Float64, String}}
└──      return %6



NamedTuple{(:x, :sin_x, :str_x), Tuple{Int64, Float64, String}}

In [37]:
function bad_arrays(xs)
    arr = [] # Cameron, you are a badman!
    for x in xs
        push!(arr, do_stuff(x))
    end
    return mapreduce(nt -> nt.sin_x, +, arr)
end

function good_arrays(xs)
    # Get the exact type of the thing you're going to store!
    first_val = do_stuff(xs[1])
    
    # Now julia can infer the type of the array
    arr = [first_val]
    for x in xs[2:end]
        push!(arr, do_stuff(x))
    end
    return mapreduce(nt -> nt.sin_x, +, arr)
end

good_arrays (generic function with 1 method)

In [38]:
xs = randn(100)
b1 = @benchmark bad_arrays($xs)
b2 = @benchmark good_arrays($xs)

display(b1)
display(b2)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  19.127 μs …  96.545 ms  ┊ GC (min … max):  0.00% … 99.91%
 Time  (median):     55.374 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   64.667 μs ± 966.960 μs  ┊ GC (mean ± σ):  14.92% ±  1.00%

           ▂█▆▂                ▃▅▃▂▁                            
  ▃▃▃▂▂▂▁▂▃██████▇▇▆▅▅▄▄▄▄▄▄▅▇███████▆▆▅▅▃▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  19.1 μs         Histogram: frequency by time          105 μs <

 Memory estimate: 50.33 KiB, allocs estimate: 503.

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  15.470 μs …  96.229 ms  ┊ GC (min … max):  0.00% … 94.36%
 Time  (median):     46.663 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   58.077 μs ± 965.600 μs  ┊ GC (mean ± σ):  15.64% ±  0.94%

          ▃█▆▂         ▂▄▃▃▁                                    
  ▂▁▁▁▂▃▃▅████▇▅▅▄▄▅▆▇███████▇▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  15.5 μs         Histogram: frequency by time          104 μs <

 Memory estimate: 48.52 KiB, allocs estimate: 205.

In [57]:
function gooder_arrays(xs)
    # Get the exact type of the thing you're going to store!
    first_val = do_stuff(xs[1])
    
    # Now julia can infer the type of the array
    arr = Array{typeof(first_val)}(undef, length(xs))
    arr[1] = first_val
    
    for i in 2:length(xs)
        arr[i] = do_stuff(xs[i])
    end
    
    return mapreduce(nt -> nt.sin_x, +, arr)
end

@benchmark gooder_arrays(xs)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  12.547 μs …  2.762 ms  ┊ GC (min … max):  0.00% … 99.19%
 Time  (median):     14.276 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   18.636 μs ± 78.285 μs  ┊ GC (mean ± σ):  13.19% ±  3.13%

  ▂▆▇██▇▅▂                            ▂▁▁▂▂▂▁▁▁               ▂
  █████████▆▆▆▇▆▆▆▇▆▆▆▅▆▇▆▆▆▆▆▆▆▆▅▆███████████████▇▆▇▇▇▆▆▇▆▆▅ █
  12.5 μs      Histogram: log(frequency) by time      36.3 μs <

 Memory estimate: 44.70 KiB, allocs estimate: 202.

In [60]:
function best_arrays(xs)
    # map can handle all the typing information on our behalf!
    arr = map(do_stuff, xs)
    return mapreduce(nt -> nt.sin_x, +, arr)
end

@benchmark best_arrays(xs)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  12.260 μs …  2.547 ms  ┊ GC (min … max):  0.00% … 98.81%
 Time  (median):     13.976 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   18.424 μs ± 77.355 μs  ┊ GC (mean ± σ):  13.19% ±  3.13%

  ▁▅▇██▇▄▁                           ▁▂▁▂▂▁▁                  ▂
  ████████▇▄▅▆▅▅▆▆▆▆▆▇▆▇▇▆▇▇▆▆▆▆▆▆▆███████████▇█▇█▇▇▇▇▇▇▆▇▆▆▆ █
  12.3 μs      Histogram: log(frequency) by time      36.6 μs <

 Memory estimate: 44.70 KiB, allocs estimate: 202.

## Abstract types and structs

We'll often have structs we write to contain results, set model configurations, etc. 

In [39]:
struct StructWithField
    a::Float64 # Could be Float32!
    b::Int
    c::String
end

Sometimes we don't know ahead of time what the type is going to be, or we are too lazy to type them. What people do in these cases is something like

In [40]:
struct BadlyTypedStruct
    a::Real
    b::Integer # Integer is an abstract type for Int
    c::Any
    d # Leaving this blank means ::Any
end

These structs end up being slow and clunky for the same reason that non-concrete arrays are slow.

The solution? Parametric typing!

## Parametric types

Parametric types are a way of saying "I know I want this struct to be typed, but I don't know it yet".

The way we do this in Julia is to write

```julia
struct TypeName{TypeVariable}
    field::TypeVariable
end
```

When we construct `TypeName` and give it an input, it will set the type `TypeVariable` to be whatever we put in `field`, which makes `TypeName` stable & concrete (provided the input is concretely typed).

In [43]:
struct ParametricStruct{A<:Real,B<:Integer,C}
    field_one::A 
    field_two::B
    field_three::C
    field_four::A # Note that this re-uses the parametric type A
end

ParametricStruct(1.0, 3, randn(10), 19.0)

ParametricStruct{Float64, Int64, Vector{Float64}}(1.0, 3, [-0.22877671217838202, -1.4487208665098041, 1.706598361648395, -0.5856710220501827, 0.20652982513934418, 0.4412579882590262, 0.9379041022797795, 0.2385759083643314, 0.2824762840263909, 1.7118707090323668], 19.0)

The performance gains from parametrically typed structs are similar to those from allowing type stability, so I won't replicate them here. 

But in general, __don't use abstract types in field definitions__. 

Either parametrically type them or statically type them (i.e. `Float64`).

## Don't change the types of things during runtime

If you assign a variable a value, try to change that variable only to other values of the same type. I.e. don't do stuff like:

In [46]:
function type_changer()
    x = "hello" # Start as a string
    for i in 1:10
        random_draw = randn()
        if random_draw >= 0
            # Changes x to an Float64!
            x = random_draw
        else
            # Add the new value as a string to the end of x
            x = string(x, random_draw)
        end
    end
    return x
end
type_changer()

0.9027571747422789

In [3]:
@code_warntype type_changer()

MethodInstance for type_changer()
  from type_changer() in Main at In[2]:1
Arguments
  #self#::Core.Const(type_changer)
Locals
  @_2::Union{Nothing, Tuple{Int64, Int64}}
  x::Union{Float64, String}
  i::Int64
  random_draw::Float64
Body::Union{Float64, String}
1 ─       (x = "hello")
│   %2  = (1:10)::Core.Const(1:10)
│         (@_2 = Base.iterate(%2))
│   %4  = (@_2::Core.Const((1, 1)) === nothing)::Core.Const(false)
│   %5  = Base.not_int(%4)::Core.Const(true)
└──       goto #7 if not %5
2 ┄ %7  = @_2::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (random_draw = Main.randn())
│   %11 = (random_draw >= 0)::Bool
└──       goto #4 if not %11
3 ─       (x = random_draw)
└──       goto #5
4 ─       (x = Main.string(x, random_draw))
5 ┄       (@_2 = Base.iterate(%2, %9))
│   %17 = (@_2 === nothing)::Bool
│   %18 = Base.not_int(%17)::Bool
└──       goto #7 if not %18
6 ─       goto #2
7 ┄       return x



In [488]:
# Or, using Traceur:
@trace type_changer()

┌ Warning: dynamic dispatch to (Core.apply_type(Base.Val, _2))()
└ @ essentials.jl:701
┌ Warning: dynamic dispatch to Base.ArgumentError("negative maxsize")
└ @ iobuffer.jl:92
┌ Warning:  is assigned as Int64
└ @ ryu/shortest.jl:313
┌ Warning:  is assigned as Nothing
└ @ ryu/shortest.jl:313
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64, Int64}}
└ @ ryu/shortest.jl:408
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64, Int64}}
└ @ ryu/shortest.jl:411
┌ Warning: c is assigned as UInt32
└ @ ryu/shortest.jl:359
┌ Warning: c is assigned as UInt32
└ @ ryu/shortest.jl:374
┌ Warning: c is assigned as UInt32
└ @ ryu/shortest.jl:383
┌ Warning: c is assigned as UInt32
└ @ ryu/shortest.jl:389
┌ Warning: c is assigned as Int64
└ @ ryu/shortest.jl:472
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64, Int64}}
└ @ ryu/shortest.jl:339
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64, Int64}}
└ @ ryu/shortest.jl:341
┌ Warning: dynamic dispatch to Base.AssertionError("0 < pos <= len

0.7006771111649364

## Break up big functions into multiple smaller functions

When you add more functions, Julia is able to specialize each function better. You get lots of free speed (and remove roadblocks for yourself) when you can let the compiler be smart for you.

It's also considered good form! It's much easier to debug one small component function than a gigantic super function.

In [50]:
function fill_random(n)
    v = rand(Bool) ? Vector{Float64}(undef, n) : Vector{Int}(undef, n)
    for i in 1:n
        v[i] = rand(eltype(v))
    end
    return v
end
# @code_warntype fill_random(100)

fill_random (generic function with 1 method)

In [48]:
@benchmark fill_random(1000)

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.591 μs …  1.383 ms  ┊ GC (min … max): 0.00% … 97.11%
 Time  (median):     6.466 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.748 μs ± 14.642 μs  ┊ GC (mean ± σ):  1.99% ±  0.97%

    █▂                                                        
  ▂▆██▄▃▃▂▂▃▃▃▃▃▄▄▄▅▅▆▆▆▅▅▄▃▃▃▂▂▂▂▁▂▁▂▁▂▁▁▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁ ▂
  3.59 μs        Histogram: frequency by time        13.8 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

In [55]:
# rfill! can be specialized for Float64 or Int!
function rfill!(v::Vector{T}) where T
    for i in eachindex(v)
        v[i] = rand(T)
    end
end

function fill_random_compartment(n)
    v = rand(Bool) ? Vector{Float64}(undef, n) : Vector{Int}(undef, n)
    rfill!(v)
    return v
end

# @code_warntype rfill!(zeros(10))

fill_random_compartment (generic function with 1 method)

In [52]:
@benchmark fill_random_compartment(1000)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.977 μs …  1.586 ms  ┊ GC (min … max): 0.00% … 97.68%
 Time  (median):     5.536 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.396 μs ± 18.051 μs  ┊ GC (mean ± σ):  2.42% ±  0.98%

  ▂     ▂▅▇██▇█▇▄▂▁                                           
  █▇▃▃▃▆███████████▇▅▅▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  2.98 μs        Histogram: frequency by time          15 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

## Memory allocations

Memory allocation can be a costly operation for your machine to do. 

When you create a new object in memory, Julia has to make or find space for that object in RAM.

## Memory allocations: the stack and the heap

There's two broad ways that memory works, commonly referred to as the __stack__ and the __heap__.

The __stack__ is for short-term memory. You can think of it like a stack of plates -- when you add to it, you put a new thing on top. When you take things off the stack, you remove them from the top as well (LIFO). The stack is for small stuff, usually.

The __heap__ is more like a bookshelf -- you don't "cover up" other things, but you may have to read the book titles to find what you're looking for. You can put huge things on the heap.



## Memory allocations: where is my thing?

Different objects are treated differently, and the compiler often has a say in whether something is heap or stack.

In general, though:

- Heap: arrays, dictionaries, mutable structs, etc.
- Stack: non-mutable structs, single values, named tuples, static arrays.

Again, this is not hard-and-fast! Any type can end up in either the stack or the heap.

## Memory allocations: why does this matter?

Heap allocation is slower than is stack allocation, so sometimes it's worth keeping in mind whether something is liable to end up on the heap.

To show you how this works, let's use `StaticArrays`, which are fixed-size arrays that the compiler usually stack-allocates.

In [56]:
# import Pkg; Pkg.add("StaticArrays")
using StaticArrays

In [57]:
function array_messabout(m)
    # Calculate product
    a = m*m'
    
    # Invert that bad boy
    b = inv(a)
    
    # Now hit me with the LU factorization
    # Why? Who knows! EdUcAtIoN iS gOoD
    return lu(b)
end

array_messabout (generic function with 1 method)

In [68]:
k = 10
m = randn(k,k)
m_static = SMatrix{k,k}(m)
@btime array_messabout($m);
@btime array_messabout($m_static);

  3.966 μs (7 allocations: 8.03 KiB)
  2.408 μs (0 allocations: 0 bytes)


The (occasional) performance gains you get from working with stack-allocated, immutable types can sometimes be amazing.

This is why I usually recommend people do __not__ use mutable structs! In some cases it's cheaper to stack-allocate a new immutable struct rather than modify a heap-allocated mutable struct.

In [58]:
struct Immutable{T}
    a::T
    b::T
    c::Matrix{T}
end

mutable struct Mutable{T}
    a::T
    b::T
    c::Matrix{T}
end

In [59]:
function double(m::Immutable)
    return Immutable(
        m.a * 2,
        m.b * 2,
        m.c * 2
    )
end

function double!(m::Mutable)
    m.a *= 2
    m.b *= 2
    m.c *= 2
    return m
end

# Because I am lazy, don't @ me
double(m::Mutable) = double!(m)

double (generic function with 2 methods)

In [60]:
function mult_double(val, n::Int) where T
    for i in 1:n
        val = double(val)
    end
    return val
end

imm = Immutable(1.0, 2.0, randn(100,100));
mut = Mutable(1.0, 2.0, randn(100,100));

@btime mult_double($imm, 1000);
@btime mult_double($mut, 1000);

  9.128 ms (2002 allocations: 76.34 MiB)
  8.546 ms (2000 allocations: 76.34 MiB)


## Column order vs. row order

When Julia stores arrays in memory, values in the same column are next to each other. In layman's terms, it's faster to go to the next row within a column rather than to go to the next column within a row.

In [61]:
function rowsum(M)
    s = zero(eltype(M))
    for row in 1:size(M,1)
        for col in 1:size(M,2)
            s += M[row, col]
        end
    end
    return s
end

function colsum(M)
    s = zero(eltype(M))
    for col in 1:size(M,2) # The only difference here is that col goes first!
        for row in 1:size(M,1)
            s += M[row, col]
        end
    end
    return s
end

colsum (generic function with 1 method)

In [62]:
M = randn(10_000, 10_000)

@btime rowsum($M);
@btime colsum($M);

  1.809 s (0 allocations: 0 bytes)
  177.955 ms (0 allocations: 0 bytes)


## Column/row order: the takeaway

When looping through columns or rows, remember that it's easier to go down a row than it is to go across a column. 

__Put rows in your inner loop, columns on your outer loop__.

## Compiler annotations

Sometimes you can give the compiler helpful hints to remove things like bounds checking, or tell the compiler that something can be cheaply parallelized at the CPU level.

`@inbounds` in front of an expression removes bounds checking. Bounds checking is there to protect you to make sure that you aren't trying to get the 11th item of a 10-element array. 

For simple loops, Julia can figure out usually that you're within bounds and won't check for you, but sometimes it can help to annotate.

It's important sometimes -- only use `@inbounds` when you know for a _fact_ that you are accessing the correct element of an array. If you screw this up, you could get silently nonsensical results.

In [64]:
function sumprod_normal(A::AbstractArray, B::AbstractArray)
    @assert size(A) == size(B)
    r = zero(eltype(A))
    for i in eachindex(A)
        r += A[i] * B[i]
    end
    return r
end

function sumprod_inbounds(A::AbstractArray, B::AbstractArray)
    @assert size(A) == size(B)
    r = zero(eltype(A))
    for i in eachindex(A)
        @inbounds r += A[i] * B[i]
    end
    return r
end

sumprod_inbounds (generic function with 1 method)

In [457]:
xs1 = randn(10_000)
xs2 = randn(10_000)
@btime sumprod_normal(xs1, xs2)
@btime sumprod_inbounds(xs1, xs2);

  19.234 μs (1 allocation: 16 bytes)
  16.116 μs (1 allocation: 16 bytes)


98.99688965175507

## `@simd`

Single instruction, multiple data is a parallelization paradigm. Essentially, you tell your processor it can do the same thing with lots of different data points, and this _can_ speed up your code.

When you use `@simd`, you are telling the compiler that the order of your operations can be done in a more or less random order. You for loop operations cannot depend on one another! No time series SIMD!

In [469]:
function sumprod_simd(A::AbstractArray, B::AbstractArray)
    @assert size(A) == size(B)
    r = zero(eltype(A))
    @simd for i in eachindex(A)
        r += A[i] * B[i]
    end
    return r
end;


In [470]:
@btime sumprod_simd($xs1, $xs2);

  15.526 μs (1 allocation: 16 bytes)


You can also combine lots of performance annotations, though __do this cautiously!__

In [472]:
function sumprod_super(A::AbstractArray, B::AbstractArray)
    @assert size(A) == size(B)
    r = zero(eltype(A))
    @simd for i in eachindex(A)
        @inbounds r += A[i] * B[i]
    end
    return r
end;

In [473]:
@btime sumprod_super($xs1, $xs2);

  2.515 μs (1 allocation: 16 bytes)


## Don't do stuff to global variables

You should not be using non-constant global variables anyway! As a refresher, `x` below is a global variable, because everything can access it.

In [476]:
x = "I am a bad global variable user" # Bad person!

function do_stuff()
    # x here is global, and the compiler 
    # cannot assert the stability of its type
    # in general cases.
    return x * ", please be kind to me"
end

@btime do_stuff();

  48.117 ns (1 allocation: 80 bytes)


There's two solutions here:

1. Annotate your global variable with `const`, which says "I am not going to change this variable, I promise".
2. Pass the variable in as an argument to the function.

In [477]:
const x_constant = "I am a bad global variable user"

function do_stuff_constant()
    # Now x is constant, the compiler can handle it.
    return x_constant * ", please be kind to me"
end

@btime do_stuff_constant();

  29.286 ns (1 allocation: 80 bytes)


In [479]:
# You can (and should) also pass it as an argument.
function do_stuff_arg(x_arg)
    return x_arg * ", please be kind to me"
end

@btime do_stuff_arg("I am a bad global variable user");

  28.767 ns (1 allocation: 80 bytes)


## In-place operations

If you do a ton of little matrix computations, it's often helpful to store results in a single preallocated buffer rather than continuously reallocate.

In Julia, a common way of doing this is with what are called __in-place operations__. In-place operations modify their input rather than returning a new object in memory.

## In-place operations

For example, rather than

```julia
X = A*B
```

we can do 

```julia
mul!(X, A, B)
```

which keeps allocations down. 

Oftentimes this is FREE SPEED, though not always. Benchmark your application to see if it actually helps.

In [489]:
function eigtimer()
    M = randn(50, 50)
    @btime eigvals(M)
    @btime eigvals!(M);
    return
end

eigtimer()

  4.279 μs (13 allocations: 4.81 KiB)
  3.808 μs (12 allocations: 3.94 KiB)


## Use multiple dispatch to your advantage

A common thing people might do in other languages is this:

In [494]:
function square(thing)
    if typeof(thing) <: Real
        return thing ^ 2
    elseif typeof(thing) <: Vector && eltype(thing) <: Real
        return thing * thing'
    end
end

@btime square(1.0)
@btime square([1.0, 2.0])

  0.041 ns (0 allocations: 0 bytes)
  111.885 ns (2 allocations: 176 bytes)


2×2 Matrix{Float64}:
 1.0  2.0
 2.0  4.0

The preferred approach is to break the typing into different functions:

In [495]:
square(thing::Real) = thing^2
square(thing::AbstractVector{<:Real}) = thing * thing'

@btime square(1.0)
@btime square([1.0, 2.0])

  0.038 ns (0 allocations: 0 bytes)
  112.008 ns (2 allocations: 176 bytes)


2×2 Matrix{Float64}:
 1.0  2.0
 2.0  4.0

This is not _necessarily_ a speed thing, but it can be. The example above doesn't actually have much in the way of speed improvements, but it is more attractive to write and easier to debug.

## Be careful with closures

A closure is a function that captures a variable outside the scope of the function. 

For example, the function below returns a new function that _closes_ over `r`.

In [505]:
function abmult(r::Int)
    if r < 0
        r = -r
    end
    f = x -> x * r # We close over r here -- Julia doesn't know that r is a fixed type here!
#     f(x) = x*r
    return f
end;
@btime abmult(3)(2);

  56.313 ns (1 allocation: 16 bytes)


In [503]:
function abmult2(r0::Int)
    r::Int = r0 # Add a type annotation to a local variable
    if r < 0
        r = -r
    end
    f = x -> x * r # Now we're closing over r, a local variable with a specific type
    return f
end
@btime abmult2(3)(2);

  4.381 ns (0 allocations: 0 bytes)


In [506]:
function abmult3(r::Int)
    if r < 0
        r = -r
    end
    f = let r = r # Or, we can use a let block to copy the outer r to the closure.
        x -> x * r
    end
    return f
end
@btime abmult3(3)(2);

  0.038 ns (0 allocations: 0 bytes)


Note that the use of the `let` block above may become a part of Julia's compiler, so you potentially won't need to do this in the future.

## Profiling

It's often hard to know how much time we are spending in different places in our program. Is there a slow line somewhere?

A common way programmers attempt to debug their code is to use a __profiler__. A profiler (a statistical one, anyway) is a tool that checks where you are in your program lots of times a second.

With the results you can look at which lines appear the most in your profile, and infer from that which are the most costly to compute.

In [67]:
# Import Profile
using Profile, LinearAlgebra

# Write a function to profile
function weird_inverse(A, n=10)
    result = deepcopy(A)
    eigen_storage = zeros(Union{ComplexF64}, n, size(A, 1))
    
    # Invert the matrix, double it, and re-invert it.
    # Add the result to A
    for i in 1:n
        result_inv = 2 * inv(result)
        result = result + inv(result_inv)
        eigen_storage[i,:] = eigvals!(result)
    end
    
    return result
end

weird_inverse (generic function with 2 methods)

In [68]:
# Sample data
A = randn(10,10) 

Profile.clear()           # Clear the previous Profile information, if it exists.
Profile.init(delay=1e-8)
weird_inverse(A)          # Run it once so you don't observe a bunch of compile information in your sample.
@profile weird_inverse(A) # Profile it with @profile
Profile.print(mincount=3) # Print out the result, get rid of the observations we see only 1,2 times.

Overhead ╎ [+additional indent] Count File:Line; Function
   ╎53  @Base/task.jl:423; (::IJulia.var"#15#18")()
   ╎ 53  @IJulia/src/eventloop.jl:8; eventloop(socket::ZMQ.Socket)
   ╎  53  @Base/essentials.jl:714; invokelatest
   ╎   53  @Base/essentials.jl:716; #invokelatest#2
   ╎    53  .../execute_request.jl:67; execute_request(socket::ZMQ.So...
   ╎     53  .../SoftGlobalScope.jl:65; softscope_include_string(m::Mo...
   ╎    ╎ 53  @Base/loading.jl:1196; include_string(mapexpr::type...
 19╎    ╎  53  @Base/boot.jl:373; eval
  1╎    ╎   34  In[67]:6; weird_inverse(A::Matrix{Floa...
   ╎    ╎    6   In[67]:12; weird_inverse(A::Matrix{Flo...
   ╎    ╎     5   ...bra/src/dense.jl:876; inv(A::Matrix{Float64})
   ╎    ╎    ╎ 3   ...lgebra/src/lu.jl:278; lu
   ╎    ╎    ╎  3   ...gebra/src/lu.jl:278; lu
   ╎    ╎    ╎   3   ...gebra/src/lu.jl:279; #lu#153
   ╎    ╎    ╎    3   ...inearAlgebra.jl:358; copy_oftype
  3╎    ╎    ╎     3   @Base/array.jl:369; copy
   ╎    ╎    5   In[67]:13; wei

In [69]:
# We can visualize this with ProfileViews
using ProfileView
ProfileView.view()

Gtk-Message: 10:57:57.592: Failed to load module "colorreload-gtk-module"
Gtk-Message: 10:57:57.592: Failed to load module "window-decorations-gtk-module"


Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint

# Fin!

Thanks for hanging out, everybody.